# 1.不包含可训练权重的简化版自注意力机制 

In [2]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

## 1.1 以计算inputs中"journey"的context vector为例

In [ ]:
'''Step1: Get the attention scores'''
query = inputs[1] #这里的query就是Transformers里面的query，就是当前待计算context vector的元素的embedding vector

attn_scores_1 = torch.matmul(query, inputs.T) #这里因为是将一个形状为(1,3)的向量与形状为(6,3)的矩阵相乘，所以需要对矩阵进行转置操作

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query) #这里现在进行点积的是两个一维向量，就不用进行转置操作

print(attn_scores_1)
print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])
tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [6]:
'''Simulate the dot product operation'''
res = 0

for idx, element in enumerate(inputs[0]):
    res += inputs[0][idx] * query[idx]

print(res)
print(torch.dot(inputs[0], query))

tensor(0.9544)
tensor(0.9544)


In [10]:
'''Step 2: Normaliza the attention scores to get the final attention weights'''
#1.A simple way
attn_weights_2_tmp = attn_scores_2  / attn_scores_2.sum()
print(attn_weights_2_tmp)
print(attn_weights_2_tmp.sum())

#2.Using softmax function 
# Using self-implemented softmax function
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim = 0)

attn_weights_2_naive = softmax_naive(attn_scores_2)
print(attn_weights_2_naive)
print(attn_weights_2_naive.sum())

# Using torch.softmax()
attn_weights_2 = torch.softmax(attn_scores_2, dim = 0)
print(attn_weights_2)
print(attn_weights_2.sum())

tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
tensor(1.0000)
tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
tensor(1.)
tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
tensor(1.)


In [11]:
'''Step 3: Comput the context vector '''
query = inputs[1]

context_vec_2 = torch.zeros(query.shape) #因为context vector的shape和token embedding vector是一样的
for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i] * x_i #就是将序列中每个token的注意力权重乘以它的embedding vector，然后再相加

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


## 1.2 计算输入序列中所有tokens的context vector

In [ ]:
'''1. Compute the un-normalized attention scores'''
attn_scores = torch.empty(inputs.shape[0], inputs.shape[0]) #这里之所以是inputs.shape[0]的原因是，对于每一个token，都要计算和其他所有token之间的注意力分数，而总的tokens的数量就是inputs.shape[0]，
# 也就是max_length的大小，输入序列越长，注意力分数的矩阵就越大，而且是二次方的增长关系，shape是max_length * max_length

#（1）使用循环的方式计算：速度慢，计算复杂度是O(n^2):
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)
print(attn_scores)

# (2) 使用矩阵相乘的方式计算：速度快
attn_scores = inputs @ inputs.T #@符号代表矩阵相乘的意思
print(attn_scores)

attn_scores = torch.matmul(inputs, inputs.T) #将inputs转置的根本原因是，
# 转置之后就能实现inputs中的每一行作为query，能够与inputs.T中的每一列相乘，作点积运算，得到最终的注意力分数结果矩阵
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])
tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])
tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6

In [ ]:
'''2. Apply normalization using softmax function'''
attn_weights = torch.softmax(attn_scores, dim = 1) # dim = 1是因为需要在列上进行计算
print(attn_weights)

print(attn_weights.sum(1)) #说明归一化成功了

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])
tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [ ]:
'''3. Comput the final context vector'''
all_context_vec = attn_weights @ inputs #这里不进行转置的原因是需要将attn_weights中的每一行的query的注意力权重，
# 分别乘到所有的token的embedding vector上，因此就不用转置了。
print(all_context_vec)

print(all_context_vec[1])
print(context_vec_2)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])
tensor([0.4419, 0.6515, 0.5683])
tensor([0.4419, 0.6515, 0.5683])


1.计算Context_vector过程的手动推导：
![](./images/计算上下文向量的过程推导.jpg)